# Considerações iniciais
Essa base de dados foi retirada de uma competição do kaggle, porém foi modificada.
# A competição original era sobre previsão de vendas de mais de 1000 lojas, por limitações de hardware e utilização de portfólio eu reduzi para 20 lojas.
#### Link da competição: www.kaggle.com/c/rossmann-store-sales/

# Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
pd.options.display.max_columns = None

In [2]:
total_lojas=20

# Leitura da base de dados

In [3]:
train = pd.read_csv('train.csv')

C:\Users\Guilherme\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Limitação a vinte lojas

Conforme dito, irei reduzir a 20 lojas por questões de performance.

In [4]:
train = train[train['Store'] <= total_lojas].reset_index(drop=True)

In [5]:
train.head(25)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


# Pré processamento

Na coluna StateHoliday há um problema de haver 0 em inteiro e 0 em string. Portanto, precisamos ajustar esse erro.

In [6]:
def trocar(x):
    #Poderia ser lambda
    if x ==0:
        return '0'
    else:
        return x
    
train['StateHoliday'] = train['StateHoliday'].map(trocar)
train['StateHoliday'].value_counts()

0    17934
a      342
b      120
c       76
Name: StateHoliday, dtype: int64

Após fazer a transformação nos dados, optei por criar uma coluna ao estilo OneHotEncoder sinalizando a informação.

In [7]:
train['StateHoliday'] = 'StateHoliday_'+ train['StateHoliday']
train = pd.concat([train.drop(columns='StateHoliday'),pd.get_dummies(train['StateHoliday'])],axis=1)
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,1,5,2015-07-31,5263,555,1,1,1,1,0,0,0
1,2,5,2015-07-31,6064,625,1,1,1,1,0,0,0
2,3,5,2015-07-31,8314,821,1,1,1,1,0,0,0
3,4,5,2015-07-31,13995,1498,1,1,1,1,0,0,0
4,5,5,2015-07-31,4822,559,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18467,16,2,2013-01-01,0,0,0,0,1,0,1,0,0
18468,17,2,2013-01-01,0,0,0,0,1,0,1,0,0
18469,18,2,2013-01-01,0,0,0,0,1,0,1,0,0
18470,19,2,2013-01-01,0,0,0,0,1,0,1,0,0


Para adicionar novas informações ao dataset, utilizarei o outro csv disponível e os juntarei. Novamente limitando o total de lojas.

In [8]:
store = pd.read_csv('store.csv')
store = store[store['Store'] <=total_lojas].reset_index(drop=True)
store.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Preenchimento dos valores nulos
Eu separei difentes classes para assim eu possa usar estratégias diferentes para valores numéricos e categóricos. Para esse caso, utilizei os valores mais frequentes.

In [9]:
float_columns=['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear']
float_inputer = SimpleImputer(strategy='most_frequent',add_indicator=False)
float_inputer.fit(store[float_columns])

categorical_columns=['PromoInterval']
categorical_inputer = SimpleImputer(strategy='most_frequent',add_indicator=False)
categorical_inputer.fit(store[categorical_columns])

store[float_columns] = float_inputer.transform(store[float_columns])
store[categorical_columns] = categorical_inputer.transform(store[categorical_columns])
store.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct"
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct"
4,5,a,a,29910.0,4.0,2015.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct"


## Transformações nas colunas StoreType e Assortment

Se utilizando novamente da estratégia para transformar valores em colunas, eu utilizo para as colunas StoreType e Assortment.

In [10]:
store = pd.concat([store.drop(columns=['StoreType','Assortment']),pd.get_dummies(store[['StoreType','Assortment']])],axis=1)
store.head(5)

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_a,StoreType_c,StoreType_d,Assortment_a,Assortment_c
0,1,1270.0,9.0,2008.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct",0,1,0,1,0
1,2,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",1,0,0,1,0
2,3,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",1,0,0,1,0
3,4,620.0,9.0,2009.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct",0,1,0,0,1
4,5,29910.0,4.0,2015.0,0,14.0,2011.0,"Jan,Apr,Jul,Oct",1,0,0,1,0


Restando apenas a coluna PromoInterval para processar, utilizo uma função simples para transformar os valores em numéricos.

In [11]:
def trocar2(x):
    if x =='Jan,Apr,Jul,Oct':
        return 1
    elif x == 'Mar,Jun,Sept,Dec':
        return 2 
    else:
        return 3
store['PromoInterval'] = store['PromoInterval'].map(trocar2)
store.head(3)

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_a,StoreType_c,StoreType_d,Assortment_a,Assortment_c
0,1,1270.0,9.0,2008.0,0,14.0,2011.0,1,0,1,0,1,0
1,2,570.0,11.0,2007.0,1,13.0,2010.0,1,1,0,0,1,0
2,3,14130.0,12.0,2006.0,1,14.0,2011.0,1,1,0,0,1,0


# Juntando os dois datasets através de um merge

In [12]:
train = train.merge(store,how='left',on='Store')

In [13]:
train['Date'] = pd.to_datetime(train['Date'])
train = train.sort_values(by=['Date','Store']).reset_index(drop=True)

Como o dataset está ordenado inversamente, por questão de costume eu troco para valores mais antigos primeiros, seguido pelo numero da loja.

## Criando uma coluna binária referente a cada loja 

In [14]:
train['Store'] = ('Store_'+train['Store'].astype(str))
aux = pd.get_dummies(train['Store'])
train = pd.concat([train.drop(columns='Store'),aux],axis=1)

Conforme dito, há 20 lojas. Eu optei por criar uma coluna binaria para cada loja.
Como método eu transformei os valores respectivos a cada loja em valor de string.

Exemplo: 5 --> '5'

Então, adicionei um texto informando o nome, passando a ser:

'5' --> 'Store_5'
 
E finalmente, criando uma coluna para cada loja.

In [15]:
train

,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_a,StoreType_c,StoreType_d,Assortment_a,Assortment_c,Store_1,Store_10,Store_11,Store_12,Store_13,Store_14,Store_15,Store_16,Store_17,Store_18,Store_19,Store_2,Store_20,Store_3,Store_4,Store_5,Store_6,Store_7,Store_8,Store_9
0,2,2013-01-01,0,0,0,0,1,0,1,0,0,1270.0,9.0,2008.0,0,14.0,2011.0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2013-01-01,0,0,0,0,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,2,2013-01-01,0,0,0,0,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,2,2013-01-01,0,0,0,0,1,0,1,0,0,620.0,9.0,2009.0,0,14.0,2011.0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2,2013-01-01,0,0,0,0,1,0,1,0,0,29910.0,4.0,2015.0,0,14.0,2011.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18467,5,2015-07-31,10231,979,1,1,1,1,0,0,0,3270.0,9.0,2009.0,0,14.0,2011.0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
18468,5,2015-07-31,8430,946,1,1,1,1,0,0,0,50.0,12.0,2005.0,1,26.0,2010.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
18469,5,2015-07-31,10071,936,1,1,1,1,0,0,0,13840.0,6.0,2010.0,1,14.0,2012.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
18470,5,2015-07-31,8234,718,1,1,1,1,0,0,0,3240.0,9.0,2009.0,1,22.0,2011.0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [16]:
def datas(df):
    df['Date'] = pd.to_datetime(df['Date'])
    
    df['sin_dia_do_mes'] = np.sin(2*np.pi*(df['Date'].dt.day)/31)
    df['cos_dia_do_mes'] = np.cos(2*np.pi*(df['Date'].dt.day)/31)

    df['sin_mes'] = np.sin(2*np.pi*(df['Date'].dt.month)/12)
    df['cos_mes'] = np.cos(2*np.pi*(df['Date'].dt.month)/12)

    return df
train = datas(train)
train.head(5)

,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_a,StoreType_c,StoreType_d,Assortment_a,Assortment_c,Store_1,Store_10,Store_11,Store_12,Store_13,Store_14,Store_15,Store_16,Store_17,Store_18,Store_19,Store_2,Store_20,Store_3,Store_4,Store_5,Store_6,Store_7,Store_8,Store_9,sin_dia_do_mes,cos_dia_do_mes,sin_mes,cos_mes
0,2,2013-01-01,0,0,0,0,1,0,1,0,0,1270.0,9.0,2008.0,0,14.0,2011.0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.201299,0.97953,0.5,0.866025
1,2,2013-01-01,0,0,0,0,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.201299,0.97953,0.5,0.866025
2,2,2013-01-01,0,0,0,0,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.201299,0.97953,0.5,0.866025
3,2,2013-01-01,0,0,0,0,1,0,1,0,0,620.0,9.0,2009.0,0,14.0,2011.0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.201299,0.97953,0.5,0.866025
4,2,2013-01-01,0,0,0,0,1,0,1,0,0,29910.0,4.0,2015.0,0,14.0,2011.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.201299,0.97953,0.5,0.866025


Para a representação das datas, além do dia da semana já informado no dataset, eu utilizo a técnica de fazer o seno e cosseno das datas, na esperança que os modelos captem suas ciclicidades.

## Separando em Train_lvl0, Train_lvl1 e Valid

Para este projeto, utilizarei um holdout simples separando o train_lvl1 e valid com 60 dias.

In [17]:
train['Date'].describe()

<ipython-input-17-419e7308995d>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  train['Date'].describe()


count                   18472
unique                    942
top       2013-01-01 00:00:00
freq                       20
first     2013-01-01 00:00:00
last      2015-07-31 00:00:00
Name: Date, dtype: object

In [18]:
train_lvl0 = train[(train['Date'] <= '2015-04-02')]

train_lvl1 = train[(train['Date'] > '2015-04-02')]
train_lvl1 = train_lvl1[train_lvl1['Date'] <= '2015-06-01']

valid = train[train['Date'] > '2015-06-01']

In [19]:
print(train_lvl0['Date'].describe())
print(train_lvl1['Date'].describe())
print(valid['Date'].describe())

count                   16072
unique                    822
top       2013-01-01 00:00:00
freq                       20
first     2013-01-01 00:00:00
last      2015-04-02 00:00:00
Name: Date, dtype: object
count                    1200
unique                     60
top       2015-04-03 00:00:00
freq                       20
first     2015-04-03 00:00:00
last      2015-06-01 00:00:00
Name: Date, dtype: object
count                    1200
unique                     60
top       2015-06-02 00:00:00
freq                       20
first     2015-06-02 00:00:00
last      2015-07-31 00:00:00
Name: Date, dtype: object


<ipython-input-19-8519f0fa923b>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(train_lvl0['Date'].describe())
<ipython-input-19-8519f0fa923b>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(train_lvl1['Date'].describe())
<ipython-input-19-8519f0fa923b>:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(valid['Date'].describe())


# Definindo a baseline

Definir a baseline é um passo muito importante, pois com elas conseguimos ter uma noção sobre como nossos modelos estão se saindo. Neste projeto, irei utilizar as vendas do dia anterior como baseline.

In [20]:
baseline = pd.DataFrame()
baseline['Date'] = train['Date']
baseline['Sales_original'] = train['Sales']
baseline['Lag_one_day'] = train['Sales'].shift(20).fillna(0)

In [21]:
aux = baseline[baseline['Date'] <= '2015-04-02'] 
from sklearn.metrics import mean_squared_error
print('Baseline train 0')
print('RMSE:',np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day'])))
print('RMSLE:',np.log(np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))))
print('Mae:',mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))

Baseline train 0
RMSE: 4248.04229886506
RMSLE: 8.354213520232149
Mae: 18045863.37294674


In [22]:
aux = baseline[baseline['Date'] > '2015-04-02'] 
aux = aux[aux['Date'] <= '2015-06-01']
from sklearn.metrics import mean_squared_error
print('Baseline train 1')
print('RMSE:',np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day'])))
print('RMSLE:',np.log(np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))))
print('Mae:',mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))

Baseline train 1
RMSE: 4893.367021455336
RMSLE: 8.495635897970296
Mae: 23945040.80666667


In [23]:
aux = baseline[baseline['Date'] > '2015-06-01'] 
from sklearn.metrics import mean_squared_error
print('Baseline valid')
print('RMSE:',np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day'])))
print('RMSLE:',np.log(np.sqrt(mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))))
print('Mae:',mean_squared_error(aux['Sales_original'],aux['Lag_one_day']))

Baseline valid
RMSE: 3866.655556593234
RMSLE: 8.260145215127364
Mae: 14951025.193333333


Iremos utilizar o RMSE como nossa baseline.

# Pré-processamento na base de teste
Apenas repetindo os passos feitos com a base de treino. A base de teste é fornecida na competição.

In [24]:
test = pd.read_csv('test.csv')

In [25]:
test = test[test['Store'] <= total_lojas].reset_index(drop=True)
test['StateHoliday'] = test['StateHoliday'].map(trocar)
test['StateHoliday'] = 'StateHoliday_'+ test['StateHoliday']
test = pd.concat([test.drop(columns='StateHoliday'),pd.get_dummies(test['StateHoliday'])],axis=1)
test = test.merge(store,how='left',on='Store')
test['Date'] = pd.to_datetime(test['Date'])
test = datas(test)
test['Store'] = ('Store_'+test['Store'].astype(str))
aux = pd.get_dummies(test['Store'])
test = pd.concat([test.drop(columns='Store'),aux],axis=1)

In [26]:
test

,Id,DayOfWeek,Date,Open,Promo,SchoolHoliday,StateHoliday_0,StateHoliday_a,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_a,StoreType_c,StoreType_d,Assortment_a,Assortment_c,sin_dia_do_mes,cos_dia_do_mes,sin_mes,cos_mes,Store_1,Store_10,Store_11,Store_12,Store_13,Store_14,Store_15,Store_16,Store_19,Store_20,Store_3,Store_7,Store_8,Store_9
0,1,4,2015-09-17,1.0,1,0,1,0,1270.0,9.0,2008.0,0,14.0,2011.0,1,0,1,0,1,0,-0.299363,-0.954139,-1.000000,-1.836970e-16,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,4,2015-09-17,1.0,1,0,1,0,14130.0,12.0,2006.0,1,14.0,2011.0,1,1,0,0,1,0,-0.299363,-0.954139,-1.000000,-1.836970e-16,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,3,4,2015-09-17,1.0,1,0,1,0,24000.0,4.0,2013.0,0,14.0,2011.0,1,1,0,0,0,1,-0.299363,-0.954139,-1.000000,-1.836970e-16,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,4,2015-09-17,1.0,1,0,1,0,7520.0,10.0,2014.0,0,14.0,2011.0,1,1,0,0,1,0,-0.299363,-0.954139,-1.000000,-1.836970e-16,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,4,2015-09-17,1.0,1,0,1,0,2030.0,8.0,2000.0,0,14.0,2011.0,1,1,0,0,0,1,-0.299363,-0.954139,-1.000000,-1.836970e-16,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,40242,6,2015-08-01,1.0,0,0,1,0,1300.0,3.0,2014.0,1,40.0,2011.0,1,1,0,0,1,0,0.201299,0.979530,-0.866025,-5.000000e-01,0,0,0,0,0,1,0,0,0,0,0,0,0,0
668,40243,6,2015-08-01,1.0,0,0,1,0,4110.0,3.0,2010.0,1,14.0,2011.0,1,0,0,1,0,1,0.201299,0.979530,-0.866025,-5.000000e-01,0,0,0,0,0,0,1,0,0,0,0,0,0,0
669,40244,6,2015-08-01,1.0,0,0,1,0,3270.0,9.0,2009.0,0,14.0,2011.0,1,1,0,0,0,1,0.201299,0.979530,-0.866025,-5.000000e-01,0,0,0,0,0,0,0,1,0,0,0,0,0,0
670,40245,6,2015-08-01,1.0,0,0,1,0,3240.0,9.0,2009.0,1,22.0,2011.0,2,1,0,0,0,1,0.201299,0.979530,-0.866025,-5.000000e-01,0,0,0,0,0,0,0,0,1,0,0,0,0,0


## Garantindo que base de teste tem as mesmas colunas que a base de treino

In [27]:
test2 = pd.DataFrame()
test2['Id'] = test['Id']
missing_cols = set( train.columns ) - set( test.columns )

for c in missing_cols:
    test[c] = 0

test = test[train.columns]
#Coluna a mais - sale
#Coluna removida - ID
test = pd.concat([test2,test.drop(columns='Sales')],axis=1)

## Salvando as bases

In [28]:
train_lvl0.to_csv('train_lvl0.csv',index=False)
train_lvl1.to_csv('train_lvl1.csv',index=False)
valid.to_csv('valid.csv',index=False)
test.to_csv('teste.csv',index=False)